In [1]:
import numpy as np
import random
import time
import pprint

from pynq import PL
from pynq import Overlay
from pynq import Clocks
from pynq import allocate


PL.reset()

ol = Overlay('./design_1.bit')

for k in ol.ip_dict:
    print(k)

Clocks.pl_clk0_mhz = 100
print(f"pl_clk0_mhz = {Clocks.pl_clk0_mhz}")

rx_dma
udp_engine_100g_ip
tx_dma
cmac_usplus_0
zynq_ultra_ps_e_0
pl_clk0_mhz = 100


In [2]:
cmac = ol.cmac_usplus_0

# Actually, you don't need to plug in the hardware
# cmac.internal_loopback = 1
# print(cmac.internal_loopback)

cmac.start()
cmac.getStats()

{'tx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 0,
  'packets_bad_fcs': 0,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 90303343}

In [3]:
from udp_engine_control import *

############################################################################################################
# Configuring FPGA Connection
############################################################################################################
my_connection_manager_sw = connection_manager_sw()
my_udp_engine_controller = udp_engine_controller(ol.udp_engine_100g_ip, my_connection_manager_sw)

MY_DST_CONFIG_MAC   =   random.getrandbits(48)
MY_SRC_CONFIG_MAC   =   random.getrandbits(48)
MY_SRC_CONFIG_IP    =   random.getrandbits(32)
MY_SRC_CONFIG_PORT  =   random.getrandbits(16)


my_udp_engine_controller.configure(MY_SRC_CONFIG_MAC, MY_SRC_CONFIG_IP, MY_SRC_CONFIG_PORT, MY_DST_CONFIG_MAC)
print("INFO:     Configured UDP Engine with:")
print(f"MY_DST_CONFIG_MAC   = {hex(MY_DST_CONFIG_MAC)}")
print(f"MY_SRC_CONFIG_MAC   = {hex(MY_SRC_CONFIG_MAC)}")
print(f"MY_SRC_CONFIG_IP    = {hex(MY_SRC_CONFIG_IP)}")
print(f"MY_SRC_CONFIG_PORT  = {hex(MY_SRC_CONFIG_PORT)}")


INFO:     Configured UDP Engine with:
MY_DST_CONFIG_MAC   = 0xef583ef20b21
MY_SRC_CONFIG_MAC   = 0x93113da67326
MY_SRC_CONFIG_IP    = 0x7000891a
MY_SRC_CONFIG_PORT  = 0x6521


In [4]:
############################################################################################################
# Filling Connection Manager
############################################################################################################

COLLISION_POOL = list(my_connection_manager_sw.generate_collision_entries())
print("SUCCESS:    Generated COLLISION_POOL")

NUM_WRITES     = 100
for _ in range(NUM_WRITES):
    pick = random.choice(COLLISION_POOL)
    bind = (random.random() < 0.9)

    if bind:
        if my_udp_engine_controller.bind_connection(pick['ip'], pick['port']):
            print(f"SUCCESS:  (bind)   ipAddr = {hex(pick['ip'])}, port = {hex(pick['port'])}")
    else:
        if my_udp_engine_controller.unbind_connection(pick['ip'], pick['port']):
            print(f"SUCCESS:  (unbind) ipAddr = {hex(pick['ip'])}, port = {hex(pick['port'])}")



SUCCESS:    Generated COLLISION_POOL
SUCCESS:  (bind)   ipAddr = 0xb8011efb, port = 0xbebb
SUCCESS:  (bind)   ipAddr = 0x60e046f1, port = 0x2564
SUCCESS:  (bind)   ipAddr = 0xc37a174d, port = 0x989f
SUCCESS:  (bind)   ipAddr = 0x5fc770a, port = 0xc975
SUCCESS:  (bind)   ipAddr = 0x3d6f4f99, port = 0x71ad
SUCCESS:  (bind)   ipAddr = 0x2d20dbde, port = 0xc4ee
SUCCESS:  (bind)   ipAddr = 0xc37a174d, port = 0x989f
SUCCESS:  (bind)   ipAddr = 0xa3a95557, port = 0x419f
SUCCESS:  (bind)   ipAddr = 0x569cf066, port = 0xf77e
SUCCESS:  (bind)   ipAddr = 0xfc51da40, port = 0x5d65
SUCCESS:  (bind)   ipAddr = 0x60e046f1, port = 0x2564
SUCCESS:  (bind)   ipAddr = 0x2d20dbde, port = 0xc4ee
SUCCESS:  (bind)   ipAddr = 0x2d20dbde, port = 0xc4ee
SUCCESS:  (bind)   ipAddr = 0x3d6f4f99, port = 0x71ad
SUCCESS:  (bind)   ipAddr = 0xbce01a1a, port = 0xa053
SUCCESS:  (bind)   ipAddr = 0x138fb575, port = 0xa2d9
SUCCESS:  (bind)   ipAddr = 0x5fc770a, port = 0xc975
SUCCESS:  (bind)   ipAddr = 0xe362827, port = 0

In [9]:
############################################################################################################
# TX Test
############################################################################################################

my_udp_model = udp_model(my_connection_manager_sw,
                            MY_DST_CONFIG_MAC,
                            MY_SRC_CONFIG_MAC,
                            MY_SRC_CONFIG_IP,
                            MY_SRC_CONFIG_PORT)

tx_dma = ol.tx_dma
rx_dma = ol.rx_dma

NUM_TESTS    = 1000
for i in range(NUM_TESTS):
    print("---------------------------------------------------------------")
    print(f"INFO:     Test Idx = {i}:\n")
    print("---------------------------------------------------------------")

    # generate test
    test_unit = my_udp_model.generate_test()
    print(f"INFO:     Generated test unit with:")
    print(f"            connection_id = {test_unit['connection_id']}")
    print(f"            input_payload_bytes_length = {test_unit['input_payload_bytes_length']}")
    print(f"            output_packet_bytes_length = {test_unit['output_packet_bytes_length']}")
    print(f"            dropped = {test_unit['dropped']}")
    
    #send through dma
    dma_buf_in = allocate(test_unit['input_payload_bytes_length'], dtype=np.uint8)
    dma_buf_in[:] = np.array(test_unit['input_payload_bytes'], dtype=np.uint8)
    tx_dma.sendchannel.transfer(dma_buf_in)
    tx_dma.sendchannel.wait()
    print("INFO:     packet sent through tx_dma")
    
    #receive response if expecting any
    if test_unit['dropped']==0:
        dma_buf_out = allocate(test_unit['output_packet_bytes_length'], dtype=np.uint8)
        rx_dma.recvchannel.transfer(dma_buf_out)
        rx_dma.recvchannel.wait()
        print("INFO:     packet received through rx_dma")

        # validate
        if np.array_equal(dma_buf_out, test_unit['output_packet_bytes']):
            print(f"SUCCESS: test {i}/{NUM_TESTS} passed")
        else:
            print(f"FAIL:    test {i}/{NUM_TESTS} failed")
            print(f"expected = {test_unit['output_packet_bytes']}")
            print(f"actual = {dma_buf_out}")


---------------------------------------------------------------
INFO:     Test Idx = 0:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 40154
            input_payload_bytes_length = 199
            output_packet_bytes_length = 241
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 0/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 1:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 154305
            input_payload_bytes_length = 218
            output_packet_bytes_length = 260
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 1/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 2:

------------------------

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 87/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 88:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 23233
            input_payload_bytes_length = 218
            output_packet_bytes_length = 260
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 88/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 89:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 250782
            input_payload_bytes_length = 80
            output_packet_bytes_length = 122
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 89/1000 passed
---

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 174/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 175:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 119710
            input_payload_bytes_length = 64
            output_packet_bytes_length = 106
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 175/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 176:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 180758
            input_payload_bytes_length = 162
            output_packet_bytes_length = 204
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 176/1000 pass

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 259/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 260:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 105690
            input_payload_bytes_length = 144
            output_packet_bytes_length = 186
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 260/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 261:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 236762
            input_payload_bytes_length = 205
            output_packet_bytes_length = 247
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 261/1000 pas

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 347/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 348:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 40154
            input_payload_bytes_length = 218
            output_packet_bytes_length = 260
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 348/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 349:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 236762
            input_payload_bytes_length = 228
            output_packet_bytes_length = 270
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 349/1000 pass

...

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 609/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 610:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 40154
            input_payload_bytes_length = 260
            output_packet_bytes_length = 302
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 610/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 611:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 54174
            input_payload_bytes_length = 71
            output_packet_bytes_length = 113
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 611/1000 passed

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 695/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 696:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 54174
            input_payload_bytes_length = 133
            output_packet_bytes_length = 175
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 696/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 697:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 154305
            input_payload_bytes_length = 256
            output_packet_bytes_length = 298
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 697/1000 pass

INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 785/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 786:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 246294
            input_payload_bytes_length = 64
            output_packet_bytes_length = 106
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 786/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 787:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 243975
            input_payload_bytes_length = 154
            output_packet_bytes_length = 196
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 787/1000 pass

INFO:     packet received through rx_dma
SUCCESS: test 870/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 871:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 185246
            input_payload_bytes_length = 86
            output_packet_bytes_length = 0
            dropped = 1
INFO:     packet sent through tx_dma
---------------------------------------------------------------
INFO:     Test Idx = 872:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 119710
            input_payload_bytes_length = 218
            output_packet_bytes_length = 260
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 872/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 873:

----------------

INFO:     packet sent through tx_dma
---------------------------------------------------------------
INFO:     Test Idx = 962:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 136103
            input_payload_bytes_length = 154
            output_packet_bytes_length = 0
            dropped = 1
INFO:     packet sent through tx_dma
---------------------------------------------------------------
INFO:     Test Idx = 963:

---------------------------------------------------------------
INFO:     Generated test unit with:
            connection_id = 171226
            input_payload_bytes_length = 222
            output_packet_bytes_length = 264
            dropped = 0
INFO:     packet sent through tx_dma
INFO:     packet received through rx_dma
SUCCESS: test 963/1000 passed
---------------------------------------------------------------
INFO:     Test Idx = 964:

-------------------------------------------------

In [10]:
cmac.getStats()

{'tx': {'packets': 872,
  'good_packets': 0,
  'bytes': 153688,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 872,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 872,
  'good_packets': 0,
  'bytes': 153688,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 872,
  'packets_bad_fcs': 872,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 5894038566}